<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/cosmogenic_examples/CRN_erosion_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for erosion rate calculations based on 10Be concentrations

Notebook last updated by Simon M Mudd 19/12/2023

This is a template you can use to calculate erosion rates from 10Be concentrations in detrital stream sediments. It uses CAIRN (https://doi.org/10.5194/esurf-4-655-2016) to make these computations.

You will need to enter your own data but otherwise just follow the template and you will soon be computing erosion rates.

## If you are on colab

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this.
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section.

## Get the climate data required for the analysis

We need to get some data that is used to compute pressure for our samples. This affects the 10Be production rate.

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP2.bin", "NCEP2.bin")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP_hgt.bin", "NCEP_hgt.bin")

## Getting topographic data of your study area from OpenTopography.org

OpenTopograhy (https://www.opentopography.org/) is a great place to download topographic data.

It has vast quantities of lidar data that you can download from the site. In addition there is an API for downloading global datasets (at a range of resolutions).

`lsdviztools` has a script that can download this global data and convert it to the file format and coordinate reference system (CRS) you need for analysis in `lsdtopotools`.

**To do this you need to have a user account at OpenTopography.** You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo

Once you have done that, you should get your API key and paste it into a text file somewhere safe (if any digital security experts want to explain how to better do this I am all ears). You will need to point to that file when calling the `lsdviztools` opentopography scraper.

If you are working in *colab* then you should upload the file into your working directory.

* You need to add the lower left and upper right hand coordinates, in lat-long, of your study area.
* You can do this by right clicking and copying coordinates on google maps
* Also set a data prefix name
* And decide on the data type: options here https://portal.opentopography.org/apidocs/#/Public/getGlobalDem, we use COP30

In [ ]:
lower_left_coords = [23.020977925808698, -110.07075635034748]
upper_right_coords = [23.339739844695995, -109.79747144601774]
Dataset_name = "baja"
source_name = "COP30"

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

study_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = lower_left_coords,
                        upper_right_coordinates = upper_right_coords,
                        prefix = Dataset_name,
                        api_key_file = your_OT_api_key_file)
study_DEM.print_parameters()
study_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_name+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## Make sure the data is in the correct place

Right, lets see what this place looks like:

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%matplotlib inline
Base_file = r_prefix
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology",dpi=500)

## Now for some parameter files for the analysis

We need a few data files to run the analysis. You probably don't need to change any of these unless you are an advanced user.

In [ ]:
CRN_param_fname = "./"+Dataset_name+"_"+source_name +"_UTM.CRNParam"
CRN_rasters_fname = "./"+Dataset_name+"_"+source_name +"_UTM_CRNRasters.csv"

with open(CRN_param_fname, 'w', encoding="utf-8") as f:
    f.write('min_slope: 0.0001\n')
    f.write('source_threshold: 12\n')
    f.write('search_radius_nodes: 4\n')
    f.write('threshold_stream_order: 2\n')
    f.write('theta_step: 30\n')
    f.write('phi_step: 30\n')
    f.write('Muon_scaling: BraucherBorchers\n')

with open(CRN_rasters_fname, 'w', encoding="utf-8") as f:
    f.write("./"+Dataset_name+"_"+source_name +"_UTM")
    f.write('\n')

## Now your cosmogenic data

Now you need to add in your cosmogenic data.

This has a very specific format.

It needs to be in a csv file, with headers:
`sample_name,sample_latitude,sample_longitude,nuclide,concentration,AMS_uncertainty,standardisation`

* The `sample_name` can be anything
* `sample_latitude` and `sample_longitude` should be obvious.
* `nuclide` needs to be `Be10` (exactly that, `10Be` will not work)
* `standardisation` needs to come from a list, if it is not recognised it will default to the `07KNSTD`

**You can have this file called anything, but I will copy and rename it here in the template**

**I will make an example file here called `ENTER_YOUR_FILENAME_HERE.csv` but when you do this yourself you should prepare your file, upload it, and change the filename below**

In [ ]:
with open('ENTER_YOUR_FILENAME_HERE.csv', 'w', encoding="utf-8") as f:
    f.write('sample_name,sample_latitude,sample_longitude,nuclide,concentration,AMS_uncertainty,standardisation\n')
    f.write('s1,23.16157025327516,-109.8028741594473,Be10,31510,9400,07KNSTD\n')
    f.write('s2,23.10241522797601,-110.0124965416167,Be10,21060,7200,07KNSTD\n')
    f.write('s2,23.260052582414716, -109.82037293760524,Be10,35060,7200,07KNSTD\n')

In [ ]:
!cat ENTER_YOUR_FILENAME_HERE.csv

In [ ]:
my_CRN_data_filename = "ENTER_YOUR_FILENAME_HERE.csv"

In [ ]:
import subprocess
new_filename = "./"+Dataset_name+"_"+source_name +"_UTM_CRNData.csv"
subprocess.run(["cp", my_CRN_data_filename, new_filename])

### We can check where the points are:

In [ ]:
import pandas as pd
CRNData_df = pd.read_csv(new_filename)
CRNData_df.head()

In [ ]:
CRNData_df["latitude"] =  CRNData_df["sample_latitude"]
CRNData_df["longitude"] =  CRNData_df["sample_longitude"]
CRNData_df.to_csv("temp_CRNData_latlong.csv")

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = Dataset_name+"_"+source_name +"_UTM"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "concentration",
                                          points_fname = "temp_CRNData_latlong.csv",
                                          scale_points = False, manual_size =50,
                                          cmap="jet", save_fig=False, size_format="geomorphology")

## Check the cosmo data

In this next step we check to see if the cosmo data is in the right place

In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"cosmo_parameter_prefix" : Dataset_name+"_"+source_name +"_UTM",
                    "check_cosmo_basins" : "true"}
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This gets the basins, which we can plot.

In [ ]:
%%capture
DataDirectory = "./"
Base_file = Dataset_name+"_"+source_name +"_UTM"
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

Before we move on, I just want to note that there are now three files that have parameters:

* `baja_COP30_UTM_CRNRasters.csv`
* `baja_COP30_UTM_CRNData.csv`
* `baja_COP30_UTM,CRNParam`

If we start to use more complex analysis, we will need to update these filenames. I will give some examples below. At the moment, we can use the `!ls` command to look at `baja_COP30_UTM_CRNRasters.csv` and see what is in there (answer: only one filename). This will change with topographic shielding and snow shielding calculations.

In [ ]:
!ls *CRNRasters.csv

## Make topographic shielding rasters

If you want to make some shielding rasters, you need to set some switches.

The examples I will do here are topographic shielding, and snow shielding.

For topographic shielding, I will actually show how to turn off topographic shielding, based on a paper by Roman DiBiase (https://esurf.copernicus.org/articles/6/923/2018/). However to implement this we still need to make a raster (that has all the topographic shielding as 1, which means no shielding at all).



In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"make_shielding_rasters" : "true",
                    "calcualte_shielding" : "false",
                    "cosmo_parameter_prefix" : Dataset_name+"_"+source_name +"_UTM"
                   }
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls *CRNRasters.csv

In [ ]:
!cat baja_COP30_UTM_Shield_CRNRasters.csv

In [ ]:
!cat baja_COP30_UTM_CRNRasters.csv

We can look at the topographic shielding raster (which will just be a raster with all the values of 1, but it is safe to check). We use the `SimpleDrape` routine in `lsdmapwrappers` which is part of `lsdviztools`.

In [ ]:
%%capture
DataDirectory = "./"
Base_file = Dataset_name+"_"+source_name +"_UTM"
Drape_prefix = Base_file+"_SH"

### Plot the snow shield raster
this_ts_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="inferno",
                                       cbar_label = "Topographic shielding (dimensionless, 1 = unshielded)",
                                       save_fig=True, size_format="geomorphology",dpi=600)

Now display the image

In [ ]:
print(this_ts_img)
from IPython.display import display, Image
display(Image(filename=this_ts_img, width=800))

## Snow shielding

You can make snow shielding rasters with a few options.

The two options are a Bilinear show shielding, where effective snow depth increases linearly up to a certain elevation, and then decreases after that. You need to set the parameters.

Or using something called the Richards method.

If you are implementing snow shielding you will need to do a bit of research into this and then calculate the parameters for one of these two methods. In this example I am just going to use the bilinear method.

**IMPORTANT**
This component is a bit underdeveloped as of lsdtt version 0.9. So it has a few annoying elements.

1. The method is case sensitive. "bilinear" will not work, you must enter "Bilinear".
2. It does not update the CRNRasters.csv file. You need to update that yourself. I will show you how to do that further below.

In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"calculate_snow_shielding" : "true",
                    "snowpack_method" : "Bilinear",
                    "snow_SlopeAscend" : "0.035",
                    "snow_SlopeDescend" : "-0.03",
                    "snow_PeakElevation" : "800",
                    "snow_PeakSnowpack" : "30",
                    "cosmo_parameter_prefix" : Dataset_name+"_"+source_name +"_UTM_Shield"
                   }
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!cat baja_COP30_UTM_Shield_CRNRasters.csv

So you've now got a snow shielding raster, but you need to update this file so the erosion rate routine knows it exits.

In [ ]:
with open('baja_COP30_UTM_Shield_CRNRasters.csv', 'w', encoding="utf-8") as f:
    f.write('./baja_COP30_UTM,./baja_COP30_UTM_SnowBL,0,./baja_COP30_UTM_SH\n')

In [ ]:
!cat baja_COP30_UTM_Shield_CRNRasters.csv

Lets just have a look at what that raster looks like using `lsdviztools`. I use a plotting routine called `SimpleDrape`:

In [ ]:
%%capture
DataDirectory = "./"
Base_file = Dataset_name+"_"+source_name +"_UTM"
Drape_prefix = Base_file+"_SnowBL"

### Plot the snow shield raster
this_ss_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="inferno",
                                       cbar_label = "Snow shielding in g/cm^2",
                                       save_fig=True, size_format="geomorphology",dpi=600)

Now display the image

In [ ]:
print(this_ss_img)
from IPython.display import display, Image
display(Image(filename=this_ss_img, width=800))

## Now lets get the erosion rates!!

### Erosion rates with full topographic shielding and no snow shielding

Now we are going to extract the erosion rates. This takes a little while since it automatically calculates topographic shielding, which is the most computationally expensive step.

**If you followed the topographic shielding or snow shielding routines above, go to the next code block!**


In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"print_production_raster" : "true",
                    "cosmo_parameter_prefix" : Dataset_name+"_"+source_name +"_UTM",
                    "calculate_erosion_rates" : "true",
                   }
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

### Erosion rates with no topographic shielding and snow shielding

Note that you have to run the code blocks to remove topographic shielding and run snow shielding for this to work. You can also run this with just topographic shielding = 1 and no snow shielding.

In [ ]:
# IMPORTANT all the parameter values must be passed as strings.
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"print_production_raster" : "true",
                    "cosmo_parameter_prefix" : Dataset_name+"_"+source_name +"_UTM_Shield",
                    "calculate_erosion_rates" : "true",
                   }
r_prefix = Dataset_name+"_"+source_name +"_UTM"
w_prefix = Dataset_name+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

## Now lets see what the samples say!

In [ ]:
import pandas as pd

data_name = "./"+Dataset_name+"_"+source_name +"_UTM_CRNResults.csv"
CRNResults_df = pd.read_csv(data_name,skiprows=3)
print(CRNResults_df[['sample_name', 'erate_mmperkyr_rho2650']])

And if you ran with the updated shielding you can comare the result:

In [ ]:
import pandas as pd

data_name = "./"+Dataset_name+"_"+source_name +"_UTM_Shield_CRNResults.csv"
CRNResults_df = pd.read_csv(data_name,skiprows=3)
print(CRNResults_df[['sample_name', 'erate_mmperkyr_rho2650']])